In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import torch
from src import utils

In [3]:
torch.cuda.is_available()

True

In [4]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from pathlib import Path
from os import getenv
from dotenv import load_dotenv

In [6]:
data = Path(getenv("DATA_PATH", "data"))

In [7]:
articles_it = utils.load_json_iter(data / Path("french_articles"))

In [8]:
print(next(articles_it)["text"])

Il tente d'incendier sa femme : elle est indemne, lui est gravement brûlé 18h31, le 01 octobre 2016 A Le mari désespéré a tenté d'incendier sa femme.@ TIZIANA FABI / AFP 0 partage Dans le but d'incendier sa femme, un homme en instance de divorce a aspergé l'appartement conjugal d'essence. Mais ce sont ses propres vêtements qui ont pris feu. 
Le retour de flamme ne s'est pas fait attendre. Un homme s'est grièvement brûlé dans le nord de l'Italie en tentant d'incendier son épouse qui voulait le quitter, rapportent samedi des médias locaux . 
Vendredi en fin d'après-midi, une violente dispute a éclaté entre les deux conjoints quadragénaires en instance de divorce, tous deux Albanais résidant à Gorgonzola, dans l'agglomération de Milan. Ne supportant pas l'idée de la séparation, le mari s'est saisi d'un bidon d'essence pour asperger l'appartement et son épouse, avant d'allumer son briquet. 
Des brûlures au 3e degré. Mais ce sont ses propres vêtements, arrosés involontairement dans son élan

In [16]:
from os import environ
from pathlib import Path

import torch
from dotenv import load_dotenv
from torch import nn
from torch.utils.data import DataLoader

from src.RRMC import discrimNN
from src.utils import JSONDataset, load_json_iter

learning_rate = 1e-3
batch_size = 64
epoch = 5

In [17]:
root = Path(".")
load_dotenv(root / Path(".env"))

data = root / Path(str(environ.get("DATA_PATH"))) / Path("french_articles")

# max_length = max(len(doc["text"]) for doc in load_json_iter(data))
# min_length = min(len(doc["text"]) for doc in load_json_iter(data))

dataset = JSONDataset(data, min_length=50, vector_length=100)

#loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True, collate_fn=lambda x: torch.tensor(x, dtype=torch.float), pin_memory=True)
loader = DataLoader(dataset, batch_size=batch_size, drop_last=True, collate_fn=lambda x: torch.tensor(x, dtype=torch.float), pin_memory=True)

""" data1 = next(loader_iter)
#print(data1)
print(data1.size())
"""
model = discrimNN(129, 10)
loss = nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

model.train()

for epo in range(epoch):
    for i, x in enumerate(loader):
        y = torch.ones(64)
        optimizer.zero_grad()

        output = model(x)
        output = loss(output, y)

        print(f"Epoch {epo}, batch {i} : loss {output}")

        output.backward()
        optimizer.step()

        #print([sum(p.data) for p in optimizer.param_groups[0]['params']])

print("Done")

Epoch 0, batch 0 : loss 0.7477473616600037
Epoch 0, batch 1 : loss 0.7403180003166199
Epoch 0, batch 2 : loss 0.7332062721252441
Epoch 0, batch 3 : loss 0.7261781692504883


KeyboardInterrupt: 